In [ ]:
from py_clob_client.client import ClobClient

client = ClobClient("https://clob.polymarket.com")  # Level 0 (no auth)

ok = client.get_ok()
time = client.get_server_time()
print(ok, time)

In [ ]:
from py_clob_client.client import ClobClient
import os
import dotenv

dotenv.load_dotenv()

HOST = "https://clob.polymarket.com"
CHAIN_ID = 137
PRIVATE_KEY = os.getenv("PRIVATE_KEY")
FUNDER = os.getenv("WALLET_ADDRESS")

client = ClobClient(
    HOST,  # The CLOB API endpoint
    key=PRIVATE_KEY,  # Your wallet's private key
    chain_id=CHAIN_ID,  # Polygon chain ID (137)
    signature_type=0,  # 1 for email/Magic wallet signatures
    funder=FUNDER,  # Address that holds your funds
)
client.set_api_creds(client.create_or_derive_api_creds())

In [ ]:
markets = client.get_markets()

markets

In [ ]:
len(markets['data'])

In [ ]:
from py_clob_client.client import ClobClient
from py_clob_client.clob_types import BookParams


TOKEN_ADDRESS = (
    "31001962798942569684009922637681930551701210305039209468573577011625144060340"
)
token_id = TOKEN_ADDRESS
mid = client.get_midpoint(token_id)
price = client.get_price(token_id, side="BUY")
book = client.get_order_book(token_id)
books = client.get_order_books([BookParams(token_id=token_id)])
print(mid, price, book.market, len(books))

In [ ]:
# client.get_address()
from py_clob_client.clob_types import BalanceAllowanceParams, AssetType

TOKEN_ADDRESS = "100485093012082864115507309596335182311230253990302215657675540934140023078330"  # sppurs
client.get_balance_allowance(
    params=BalanceAllowanceParams(
        signature_type=0,
        asset_type=AssetType.CONDITIONAL,
        token_id=TOKEN_ADDRESS,
    )
)
# client.get_address()
# @dataclass
# class BalanceAllowanceParams:
#     asset_type: AssetType = None
#     token_id: str = None
#     signature_type: int = -1
# client.builder.sig_type

In [ ]:
client.get_address()

In [ ]:
from py_clob_client.client import ClobClient
from py_clob_client.clob_types import MarketOrderArgs, OrderType
from py_clob_client.order_builder.constants import BUY


mo = MarketOrderArgs(
    token_id=TOKEN_ADDRESS,
    amount=1,
    side=BUY,
    order_type=OrderType.FOK,
)  # Get a token ID: https://docs.polymarket.com/developers/gamma-markets-api/get-markets
signed = client.create_market_order(mo)
resp = client.post_order(signed, OrderType.FOK)
print(resp)

In [ ]:
type(resp)

In [ ]:
# client.get_last_trade_price(TOKEN_ADDRESS)
client.get_trades()

In [ ]:
from web3 import Web3
from web3.constants import MAX_INT
from web3.middleware import geth_poa_middleware
import os
import dotenv

dotenv.load_dotenv()
print(os.getenv("WALLET_ADDRESS"))

rpc_url = os.getenv("POLYGON_RPC")  # Polygon rpc url
priv_key = os.getenv("PRIVATE_KEY")  # Polygon account private key
pub_key = Web3.to_checksum_address(
    os.getenv("WALLET_ADDRESS")
)  # Polygon account public key corresponding to private key

chain_id = 137

erc20_approve = """[{"constant": false,"inputs": [{"name": "_spender","type": "address" },{ "name": "_value", "type": "uint256" }],"name": "approve","outputs": [{ "name": "", "type": "bool" }],"payable": false,"stateMutability": "nonpayable","type": "function"}]"""
erc1155_set_approval = """[{"inputs": [{ "internalType": "address", "name": "operator", "type": "address" },{ "internalType": "bool", "name": "approved", "type": "bool" }],"name": "setApprovalForAll","outputs": [],"stateMutability": "nonpayable","type": "function"}]"""

usdc_address = "0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174"  # MATIC collateral
ctf_address = "0x4D97DCd97eC945f40cF65F87097ACe5EA0476045"  # MATIC conditionalTokens

web3 = Web3(Web3.HTTPProvider(rpc_url))
web3.middleware_onion.inject(geth_poa_middleware, layer=0)

nonce = web3.eth.get_transaction_count(pub_key)

usdc = web3.eth.contract(address=usdc_address, abi=erc20_approve)
ctf = web3.eth.contract(address=ctf_address, abi=erc1155_set_approval)


# CTF Exchange
raw_usdc_approve_txn = usdc.functions.approve(
    "0x4bFb41d5B3570DeFd03C39a9A4D8dE6Bd8B8982E", int(MAX_INT, 0)
).build_transaction({"chainId": chain_id, "from": pub_key, "nonce": nonce})
signed_usdc_approve_tx = web3.eth.account.sign_transaction(
    raw_usdc_approve_txn, private_key=priv_key
)
send_usdc_approve_tx = web3.eth.send_raw_transaction(
    signed_usdc_approve_tx.raw_transaction
)
usdc_approve_tx_receipt = web3.eth.wait_for_transaction_receipt(
    send_usdc_approve_tx, 600
)
print(usdc_approve_tx_receipt)

nonce = web3.eth.get_transaction_count(pub_key)

raw_ctf_approval_txn = ctf.functions.setApprovalForAll(
    "0x4bFb41d5B3570DeFd03C39a9A4D8dE6Bd8B8982E", True
).build_transaction({"chainId": chain_id, "from": pub_key, "nonce": nonce})
signed_ctf_approval_tx = web3.eth.account.sign_transaction(
    raw_ctf_approval_txn, private_key=priv_key
)
send_ctf_approval_tx = web3.eth.send_raw_transaction(
    signed_ctf_approval_tx.raw_transaction
)
ctf_approval_tx_receipt = web3.eth.wait_for_transaction_receipt(
    send_ctf_approval_tx, 600
)
print(ctf_approval_tx_receipt)

nonce = web3.eth.get_transaction_count(pub_key)

# Neg Risk CTF Exchange
raw_usdc_approve_txn = usdc.functions.approve(
    "0xC5d563A36AE78145C45a50134d48A1215220f80a", int(MAX_INT, 0)
).build_transaction({"chainId": chain_id, "from": pub_key, "nonce": nonce})
signed_usdc_approve_tx = web3.eth.account.sign_transaction(
    raw_usdc_approve_txn, private_key=priv_key
)
send_usdc_approve_tx = web3.eth.send_raw_transaction(
    signed_usdc_approve_tx.raw_transaction
)
usdc_approve_tx_receipt = web3.eth.wait_for_transaction_receipt(
    send_usdc_approve_tx, 600
)
print(usdc_approve_tx_receipt)

nonce = web3.eth.get_transaction_count(pub_key)

raw_ctf_approval_txn = ctf.functions.setApprovalForAll(
    "0xC5d563A36AE78145C45a50134d48A1215220f80a", True
).build_transaction({"chainId": chain_id, "from": pub_key, "nonce": nonce})
signed_ctf_approval_tx = web3.eth.account.sign_transaction(
    raw_ctf_approval_txn, private_key=priv_key
)
send_ctf_approval_tx = web3.eth.send_raw_transaction(
    signed_ctf_approval_tx.raw_transaction
)
ctf_approval_tx_receipt = web3.eth.wait_for_transaction_receipt(
    send_ctf_approval_tx, 600
)
print(ctf_approval_tx_receipt)

nonce = web3.eth.get_transaction_count(pub_key)

# Neg Risk Adapter
raw_usdc_approve_txn = usdc.functions.approve(
    "0xd91E80cF2E7be2e162c6513ceD06f1dD0dA35296", int(MAX_INT, 0)
).build_transaction({"chainId": chain_id, "from": pub_key, "nonce": nonce})
signed_usdc_approve_tx = web3.eth.account.sign_transaction(
    raw_usdc_approve_txn, private_key=priv_key
)
send_usdc_approve_tx = web3.eth.send_raw_transaction(
    signed_usdc_approve_tx.raw_transaction
)
usdc_approve_tx_receipt = web3.eth.wait_for_transaction_receipt(
    send_usdc_approve_tx, 600
)
print(usdc_approve_tx_receipt)

nonce = web3.eth.get_transaction_count(pub_key)

raw_ctf_approval_txn = ctf.functions.setApprovalForAll(
    "0xd91E80cF2E7be2e162c6513ceD06f1dD0dA35296", True
).build_transaction({"chainId": chain_id, "from": pub_key, "nonce": nonce})
signed_ctf_approval_tx = web3.eth.account.sign_transaction(
    raw_ctf_approval_txn, private_key=priv_key
)
send_ctf_approval_tx = web3.eth.send_raw_transaction(
    signed_ctf_approval_tx.raw_transaction
)
ctf_approval_tx_receipt = web3.eth.wait_for_transaction_receipt(
    send_ctf_approval_tx, 600
)
print(ctf_approval_tx_receipt)